<a href="https://colab.research.google.com/github/hanifroyyanramadhan/praktikum_pembelajaran_mesin/blob/main/cnn_quality_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

* Cacat adalah hal yang tidak diinginkan dalam industri casting. Untuk menghapus produk yang rusak ini, semua industri memiliki departemen inspeksi kualitasnya. 
* Tapi, masalah utama adalah proses inspeksi ini dilakukan secara manual dan ini adalah proses yang sangat memakan waktu dan karena keterlibatan manusia.
* hasil yang diperoleh melalui metode ini tidak 100% akurat. 
* Ini bisa karena penolakan seluruh pesanan sehingga menciptakan kerugian bagi perusahaan.


## Tujuan
* Untuk mengotomatiskan proses ini menggunakan model architecture CNN.

## Dataset
* Dataset ini adalah casting produk manufaktur.
* Casting adalah proses manufaktur di mana bahan cair biasanya dituangkan ke dalam cetakan, yang berisi rongga berongga dari bentuk yang diinginkan, dan kemudian dibiarkan mengeras.
* Alasan untuk mengumpulkan data ini adalah cacat casting !!
* Cacat pengecoran adalah ketidakteraturan yang tidak diinginkan dalam proses pengecoran logam.

## Import Libraries
Seperti biasa, sebelum kita memulai analisis dan pemodelan, mari kita impor beberapa perpustakaan yang diperlukan untuk bekerja dengan data.

In [ ]:
# Data Analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Neural Network Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint

# Evaluation
from sklearn.metrics import confusion_matrix, classification_report

## Load the Images

Berikut adalah struktur folder kami yang berisi data gambar:


```
casting_data
├───test
│   ├───def_front
│   └───ok_front
└───train
    ├───def_front
    └───ok_front
```
Folder casting_data terdiri dari dua subfolder test dan train di mana masing-masing memiliki subfolder lain: def_front dan ok_front yang menunjukkan kelas variabel target kami. Gambar di dalam kereta akan digunakan untuk pemasangan model dan validasi, sementara pengujian akan digunakan murni untuk menguji kinerja model pada gambar yang tidak terlihat.

## Data Augmentation

Kami menerapkan augmentasi data on-the-fly, teknik untuk memperluas ukuran dataset pelatihan dengan membuat versi modifikasi dari gambar asli yang dapat meningkatkan kinerja model dan kemampuan untuk menggeneralisasi. Kami akan menggunakan dengan parameter berikut:

- `rotation_range`: Rentang derajat untuk rotasi acak. Kami memilih 360 derajat karena produk adalah objek bulat.
- `width_shift_range`: Rentang pecahan dari lebar total yang akan digeser.
- `height_shift_range`: Rentang fraksi dari total tinggi yang akan digeser.
- `shear_range`: Rentang derajat untuk geser acak dalam arah berlawanan arah jarum jam.
- `zoom_range`: Rentang pecahan untuk zoom acak.
- `horizontal_flip` dan `vertical_flip` diatur ke True untuk membalik gambar secara acak secara horizontal dan vertikal.
- `brightness_range`: Rentang pecahan untuk memilih nilai pergeseran kecerahan.

Parameter lainnya:

- `rescale`: Scala nilai piksel berada di rentang 0 dan 1.
- `validation_split`: Reserve 20% of the training data for validation, and the rest 80% for model fitting.

In [ ]:
train_generator = ImageDataGenerator(rotation_range = 360,
                                     width_shift_range = 0.05,
                                     height_shift_range = 0.05,
                                     shear_range = 0.05,
                                     zoom_range = 0.05,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     brightness_range = [0.75, 1.25],
                                     rescale = 1./255,
                                     validation_split = 0.2)

Kami menentukan satu set nilai lain untuk parameter 'flow_from_directory':
- `IMAGE_DIR`: Direktori tempat data gambar disimpan.
- `IMAGE_SIZE`: Dimensi gambar (300 px kali 300 px).
- `BATCH_SIZE`: Jumlah gambar yang akan dimuat dan dilatih pada satu waktu.
- `SEED_NUMBER`: Pastikan reproduktifitas.
- `color_mode = "grayscale"`: Perlakukan gambar kami hanya dengan satu warna saluran.
- `class_mode` dan `classes` menentukan kelas target dari masalah kita. Dalam hal ini, kami menunjukkan kelas 'cacat' sebagai positif (1), dan 'ok' sebagai kelas negatif.
- 'shuffle' = Benar untuk memastikan model mempelajari cacat dan gambar ok secara bergantian.

In [ ]:
IMAGE_DIR = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/"
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 64
SEED_NUMBER = 123

gen_args = dict(target_size = IMAGE_SIZE,
                color_mode = "grayscale",
                batch_size = BATCH_SIZE,
                class_mode = "binary",
                classes = {"ok_front": 0, "def_front": 1},
                seed = SEED_NUMBER)

train_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + "train",
                                        subset = "training", shuffle = True, **gen_args)
validation_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + "train",
                                        subset = "validation", shuffle = True, **gen_args)

In [ ]:
test_generator = ImageDataGenerator(rescale = 1./255)
test_dataset = test_generator.flow_from_directory(directory = IMAGE_DIR + "test",
                                                  shuffle = False,
                                                  **gen_args)

## Visualize the Image

We successfully load and apply on-the-fly data augmentation according to the specified parameters. Now, in this section, we visualize the image to make sure that it is loaded correctly.

### Visualize Image in Batch
Visualize the first batch (`BATCH_SIZE = 64`) of the training dataset (images with data augmentation) and also the test dataset (images without data augmentation).

In [ ]:
mapping_class = {0: "ok", 1: "defect"}
mapping_class

In [ ]:
def visualizeImageBatch(dataset, title):
    images, labels = next(iter(dataset))
    images = images.reshape(BATCH_SIZE, *IMAGE_SIZE)
    fig, axes = plt.subplots(8, 8, figsize=(16,16))

    for ax, img, label in zip(axes.flat, images, labels):
        ax.imshow(img, cmap = "gray")
        ax.axis("off")
        ax.set_title(mapping_class[label], size = 20)

    plt.tight_layout()
    fig.suptitle(title, size = 30, y = 1.05, fontweight = "bold")
    plt.show()
    
    return images

In [ ]:
train_images = visualizeImageBatch(train_dataset,
                                   "FIRST BATCH OF THE TRAINING IMAGES\n(WITH DATA AUGMENTATION)")

In [ ]:
test_images = visualizeImageBatch(test_dataset,
                                  "FIRST BATCH OF THE TEST IMAGES\n(WITHOUT DATA AUGMENTATION)")

### Visualize Detailed Image
Let's also take a look on the detailed image by each pixel. Instead of plotting 300 pixels by 300 pixels (which computationally expensive), we take a small part of 25 pixels by 25 pixels only

In [ ]:
img = np.squeeze(train_images[4])[75:100, 75:100]

fig = plt.figure(figsize = (15, 15))
ax = fig.add_subplot(111)
ax.imshow(img, cmap = "gray")
ax.axis("off")

w, h = img.shape
for x in range(w):
    for y in range(h):
        value = img[x][y]
        ax.annotate("{:.2f}".format(value), xy = (y,x),
                    horizontalalignment = "center",
                    verticalalignment = "center",
                    color = "white" if value < 0.4 else "black")

These are the example of values that we are going to feed into our CNN architecture.

## Training the Network

As mentioned earlier, we are going to train a CNN model to classify the casting product image. CNN is used as an automatic feature extractor from the images so that it can learn how to distinguish between `defect` and `ok` casted products. It effectively uses the adjacent pixel to downsample the image and then use a prediction (fully-connected) layer to solve the classification problem. This is a simple illustration by [Udacity](https://github.com/udacity/deep-learning-v2-pytorch) on how the layers are arranged sequentially:

![](https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/convolutional-neural-networks/conv-visualization/notebook_ims/CNN_all_layers.png)

### Define Architecture

Here is the detailed architecture that we are going to use:

1. **First convolutional layer**: consists of 32 filters with kernel_size matrix 3 by 3. Using 2-pixel strides at a time, reduce the image size by half.
2. **First pooling layer**: Using max-pooling matrix 2 by 2 (pool_size) and 2-pixel strides at a time further reduce the image size by half.
3. **Second convolutional layer**: Just like the first convolutional layer but with 16 filters only.
4. **Second pooling layer**: Same as the first pooling layer.
5. **Flattening**: Convert two-dimensional pixel values into one dimension, so that it is ready to be fed into the fully-connected layer.
6. **First dense layer + Dropout**: consists of 128 units and 1 bias unit. Dropout of rate 20% is used to prevent overfitting.
7. **Second dense layer + Dropout**: consists of 64 units and 1 bias unit. Dropout of rate 20% is also used to prevent overfitting.
8. **Output layer**: consists of only one unit and activation is a sigmoid function to convert the scores into a probability of an image being defect.

For every layer except output layer, we use Rectified Linear Unit (ReLU) activation function.

![relu](https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/convolutional-neural-networks/conv-visualization/notebook_ims/relu_ex.png)

In [ ]:
model_cnn = Sequential(
    [
        # First convolutional layer
        Conv2D(filters = 32,
               kernel_size = 3,
               strides = 2,
               activation = "relu",
               input_shape = IMAGE_SIZE + (1, )),
        
        # First pooling layer
        MaxPooling2D(pool_size = 2,
                     strides = 2),
        
        # Second convolutional layer
        Conv2D(filters = 16,
               kernel_size = 3,
               strides = 2,
               activation = "relu"),
        
        # Second pooling layer
        MaxPooling2D(pool_size = 2,
                     strides = 2),
        
        # Flattening
        Flatten(),
        
        # Fully-connected layer
        Dense(128, activation = "relu"),
        Dropout(rate = 0.2),
        
        Dense(64, activation = "relu"),
        Dropout(rate = 0.2),
        
        Dense(1, activation = "sigmoid")
    ]
)

model_cnn.summary()

### Compile the Model

Next, we specify how the model backpropagates or update the weights after each batch feed-forward. We use `adam` optimizer and a loss function `binary cross-entropy` since we are dealing with binary classification problem. The metrics used to monitor the training progress is accuracy.

In [ ]:
model_cnn.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

## Model Fitting¶
Before we do model fitting, let's check whether GPU is available or not.

In [ ]:
checkpoint = ModelCheckpoint('model/cnn_casting_inspection_model.hdf5',
                             verbose = 1,
                             save_best_only = True,
                             monitor='val_loss',
                             mode='min')

model_cnn.fit(train_dataset,
                    validation_data = validation_dataset,
                    batch_size = 16, 
                    epochs = 15,
                    callbacks = [checkpoint],
                    verbose = 1)

### Training Evaluation
Let's plot both loss and accuracy metrics for train and validation data based on each epoch.

In [ ]:
plt.subplots(figsize = (8, 6))
sns.lineplot(data = pd.DataFrame(model_cnn.history.history,
                                 index = range(1, 1+len(model_cnn.history.epoch))))
plt.title("TRAINING EVALUATION", fontweight = "bold", fontsize = 20)
plt.xlabel("Epochs")
plt.ylabel("Metrics")

plt.legend(labels = ['val loss', 'val accuracy', 'train loss', 'train accuracy'])
plt.show()

We can conclude that the model is **not overfitting** the data since both train loss and val loss simultaneously dropped towards zero. Also, both train accuracy and val accuracy increase towards 100%.

### Testing on Unseen Images

Our model performs very well on the training and validation dataset which uses augmented images. Now, we test our model performance with unseen and unaugmented images.

In [ ]:
best_model = load_model("model/cnn_casting_inspection_model.hdf5")

In [ ]:
y_pred_prob = best_model.predict(test_dataset)

The output of the prediction is in the form of probability. We use THRESHOLD = 0.5 to separate the classes. If the probability is greater or equal to the THRESHOLD, then it will be classified as defect, otherwise ok.

In [ ]:
THRESHOLD = 0.5
y_pred_class = (y_pred_prob >= THRESHOLD).reshape(-1,)
y_true_class = test_dataset.classes[test_dataset.index_array]

pd.DataFrame(
    confusion_matrix(y_true_class, y_pred_class),
    index = [["Actual", "Actual"], ["ok", "defect"]],
    columns = [["Predicted", "Predicted"], ["ok", "defect"]],
)

In [ ]:
print(classification_report(y_true_class, y_pred_class, digits = 4))

According to the problem statement, we want to minimize the case of False Negative, where the defect product is misclassified as `ok`. This can cause the whole order to be rejected and create a big loss for the company. Therefore, in this case, we prioritize Recall over Precision.

But if we take into account the cost of re-casting a product, we have to minimize the case of False Positive also, where the ok product is misclassified as `defect`. Therefore we can prioritize the `F1 score` which combines both Recall and Precision.

On test dataset, the model achieves a very good result as follow:

- Accuracy: 99.02 %
- Recall: 99.12%
- Precision: 99.34%
- F1 score 99.23%%

___

# Conclusion
By using CNN and on-the-fly data augmentation, the performance of our model in training, validation, and test images is almost perfect, reaching 98-99% accuracy and F1 score. We can utilize this model by embedding it into a surveillance camera where the system can automatically separate defective product from the production line. This method surely can reduce human error and human resources on manual inspection, but it still needs supervision from human since the model is not 100% correct at all times.